![open-cv inbuilt dataset](img/06_digits-1024x512.jpeg)

We are going to use the above image as our dataset that comes with OpenCV samples. 

It contains 5000 images in all — 500 images of each digit. 

Each image is 20×20 grayscale with a black background. 

4500 of these digits will be used for training and the remaining 500 will be used for testing the performance of the algorithm.

### Step 1: Deskewing (Pre-Processing)

People often think of a learning algorithm as a block box.

In reality, you can assist the algorithm a bit and notice huge gains in performance. For example, if you are building a face recognition system, aligning the images to a reference face often leads to a quite substantial improvement in performance.

Aligning digits before building a classifier similarly produces superior results. In the case of faces, aligment is rather obvious — you can apply a similarity transformation to an image of a face to align the two corners of the eyes to the two corners of a reference face.

In the case of handwritten digits, an obvious variation in writing among people is the slant of their writing. Some writers have a right or forward slant where the digits are slanted forward, some have a backward or left slant, and some have no slant at all. We can help the algorithm quite a bit by fixing this vertical slant so it does not have to learn this variation of the digits. The image on the left shows the original digit in the first column and it’s deskewed (fixed) version.

This deskewing of simple grayscale images can be achieved using image moments (an image moment is a certain particular weighted average (moment) of the image pixels' intensities, or a function of such moments, usually chosen to have some attractive property or interpretation.) - [Raw Moments, Central Moments, Moment Invariants]. 
OpenCV has an implementation of moments and it comes in handy while calculating useful information like centroid, area, skewness of simple images with black backgrounds.


It turns out that a measure of the skewness is the given by the ratio of the two central moments ( mu11 / mu02 ). The skewness thus calculated can be used in calculating an affine transform that deskews the image.

@Affine Tranformation:-
    1. Origin doesn't necessary map to origin
    2. Lines map to lines.
    3. Parallel lines remain parallel
    4. Ratio are preserved



In [ ]:
        def deskew(img):
            m = cv2.moments(img)
            if abs(m['mu02']) < 1e-2:
                # no deskewing needed. 
                return img.copy()
            #// Calculate skew based on central momemts. 
            skew = m['mu11']/m['mu02']
            #// Calculate affine transform to correct skewness. 
            M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
            #// Apply affine transform
            img = cv2.warpAffine(img, M, (SZ, SZ), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
            return img

### Step 2 : Calculate the Histogram of Oriented Gradients (HOG) descriptor

Convert the grayscale image to a feature vector using the HOG feature descriptor.

Gathering information is easy but the difficult part is putting that knowledge into Practise. 

Part of the reason was that a lot of these algorithms worked after tedious handtuning and it was not obvious how to set the right parameters. For example, in Harris corner detector, why is the free parameter k set to 0.04 ? Why not 1 or 2 or 0.34212 instead? Why is 42 the answer to life, universe, and everything?

As I got more real world experience, I realized that in some cases you can make an educated guess but in other cases, nobody knows why. People often do a parameter sweep — they change different parameters in a principled way to see what produces the best result. Sometimes, the best parameters have an intuitive explanation and sometimes they don’t.


    
        

In [ ]:
winSize = (20,20) ##set to 20×20 (size of the digit images) & want to calculate 1 descriptor for the entire image.
    
    ## The notion of blocks exist to tackle illumination variation.
    ## A large block size makes local changes less significant while a smaller block size weights local changes more.
    ## Typically blockSize is set to 2 x cellSize, but here illumination does not present much of a challenge. 
    ## In these experiments, a blockSize of 10×10 gave the best results.
    blockSize = (10,10)
    
    ## The blockStride determines the overlap between neighboring blocks and controls the degree of contrast normalization.
    ## Typically a blockStride is set to 50% of blockSize.
    blockStride = (5,5)
    
    ## The cellSize is chosen based on the scale of the features important to do the classification. 
    ## A very small value would blow up the size of feature vector & very large one may not capture relevant information.
    ## 8 value could have been used.
    cellSize = (10,10) 
    
    ## nbins sets the number of bins in the histogram of gradients. 
    ## HOG paper had recommended a value of 9 to capture gradients between 0 and 180 degrees in 20 degrees increments.
    nbins = 9
    
    
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    
    ## Typically gradients can have any orientation between 0 and 360 degrees referred as "Singed Gradients" 
    ## opposed to this is “unsigned” gradients that drop the sign and take values between 0 and 180 degrees. 
    ## In the original HOG paper, unsigned gradients were used for pedestrian detection. 
    ## In this experiments, for this problem, signed gradients produced slightly better results.
    signedGradients = True
    
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType, L2HysThreshold,gammaCorrection,nlevels, useSignedGradients)
    
    
    descriptor = hog.compute(im)

.

### Step 3 : Training a Model ( a.k.a Learning a Classifier ) --- C vs Gamma

Until this point, we have deskewed the original image and defined a descriptor for our image. This has allowed us to convert every image in our dataset to a vector of size 81×1.


#### SVM Parameter C
One of the two common parameters you need to know about while training an SVM is called C. Real world data is not as clean as shown above. Sometimes the training data may have mislabeled examples. At other times, one example of a set may be too close in appearance to another example. E.g. a handwritten digit 2 may look like a 3.


In the animation below we have created this scenario. Notice, the blue dot is too close to the red cluster. When the default value of C = 1 is chosen, the blue dot is misclassified. Choosing the value of 100 for C classifies it correctly.

![misclassification tradeoff in SVM](img/007_SVM-C.gif)

But now the decision boundary represented by the black line is too close to one of the classes. Would you rather choose C to be 1 where one data point is misclassified, but the separation between the classes is much better ( minus the one data point )? The parameter C allows you to control this tradeoff.

So, how do you choose C? We choose the C that provides the best classification on a held out test set. The images in this set were not used in training.

#### SVM Parameter Gamma : Non-Linear SVM
What if the data is not linearly separable? The figure below shows two classes using red and blue dots that are not linearly separable. You cannot draw a line on the plane to separate the two classes. A good classifier, represented using the black line, is more of a circle.

![non-linear SVM](img/008_non-linearly-separable-data.jpg)


In real life, data is messy and not linearly separable. Can we still use SVMs? The answer is YES!

To accomplish this, you use a technique called the Kernel Trick. It is a neat trick that transforms non-linearly separable data into a linearly separable one. In our example, the red and blue dots lie on a 2D plane. Let us add a third dimension to all data points using the following equation.

![add 3rd dimension](img/009_quicklatex.com-9c202e4a67a82ddf3e8365d052d6d46f_l3.png)

The parameter Gamma ( \gamma ) controls the stretching of data in the third dimension. It helps in classification but it also distorts the data.

If you ever hear people using the fancy term Radial Basis Function (RBF) with a Gaussian Kernel, they are simply talking about the above equation. RBF is simply a real-valued function that depends only on the distance from the origin

So, we just cooked up a third dimension based on data in the other two dimensions. The figure below shows this three-dimensional (x, y, z) data. We can see it is separable by the plane containing the black circle!

![kernel function implementation](img/010_kernel-trick.jpg)




In [ ]:
        #/ Set up SVM for OpenCV 3
        svm = cv2.ml.SVM_create()
        #/ Set SVM type
        svm.setType(cv2.ml.SVM_C_SVC)
        #/ Set SVM Kernel to Radial Basis Function (RBF) 
        svm.setKernel(cv2.ml.SVM_RBF)
        #/ Set parameter C
        svm.setC(C)
        #/ Set parameter Gamma
        svm.setGamma(gamma)

        #/ Train SVM on training data  
        svm.train(trainData, cv2.ml.ROW_SAMPLE, trainLabels)

        #/ Save trained model 
        svm->save("digits_svm_model.yml");

        #/ Test on a held out test set
        testResponse = svm.predict(testData)[1].ravel()

.

### Auto Training SVM

As you can imagine, it can be very time consuming to select the right SVM parameters C and Gamma. Fortunately, OpenCV 3.x 
### C++ API 
provides a function that automatically does this hyperparameter optimization for you and provides the best C and Gamma values.

svm->trainAuto(td);

This training can take a very long time ( say 5x more than svm->train ) because it is essentially training multiple times.



.

### OpenCV SVM bugs

We encountered two bugs while working with OpenCV SVM. The first one is confirmed, but the other two are not.

    1. SVM model won’t load in Python API. The trained SVM model you just saved won’t load if you are using Python! Is the bug fix coming ? Nope! Check it out here
    2. trainAuto does not appear to be exposed via the Python API.
    3. SVM with RBF kernel does not work in iOS / Android. I would be happy to be proven wrong, but on mobile platforms ( iOS / Android ), we have not been able to use the SVM trained with RBF kernel. The SVM response is always the same. Linear SVM models work just fine.
    
